In [163]:
import numpy as np
import time
from copy import deepcopy
import pprint

# Import Qiskit classes
import qiskit
import qiskit.quantum_info as qi
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.providers.aer import noise
from qiskit.compiler import assemble

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter
from qiskit.ignis.verification.tomography import gateset_tomography_circuits, GatesetTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

# Auxiliary methods
from qiskit.quantum_info import Choi, Kraus
from qiskit.extensions import HGate, XGate


## Custom Gates to implement simple jump, quantum jump and quantum merge operations.

In [224]:
from qiskit import transpile

Usplit = [[1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1.j, 0, 0, 0],
         [0, 1.j / np.sqrt(2), 1 / np.sqrt(2), 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, -1 / np.sqrt(2), 1.j / np.sqrt(2), 0],
         [0, 1.j / np.sqrt(2), -1 / np.sqrt(2), 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1.j / np.sqrt(2), -1 / np.sqrt(2), 0],
         [0, 0, 0, 1.j, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1]]

Ujump = [[1, 0, 0, 0],
         [0, 0, 1.j, 0],
         [0, 1.j, 0, 0],
         [0, 0, 0, 1]]
         
Umerge=np.matrix(Usplit).getH()


Usplit_fixed = []

for row in range(64):
    temp = []

    for col in range(64):
        temp.append(0)
        
        if row == col:
            temp[-1] = 1


    Usplit_fixed.append(temp)



Usplit_fixed[0][0] = 1


Usplit_fixed[2][2] = 0

Usplit_fixed[3][3] = 0

Usplit_fixed[60][60] = 0

Usplit_fixed[61][61] = 0


Usplit_fixed[8][2] = -1 / np.sqrt(2)

Usplit_fixed[12][3] = 1 / np.sqrt(2)

Usplit_fixed[32][2] = 1 / np.sqrt(2)

Usplit_fixed[48][3] = -1 / np.sqrt(2)


Usplit_fixed[63][63] = 1

Usplit_fixed[63-8][63-2] = -1 / np.sqrt(2)

Usplit_fixed[63-12][63-3] = 1 / np.sqrt(2)

Usplit_fixed[63-32][63-2] = 1 / np.sqrt(2)

Usplit_fixed[63-48][63-3] = -1 / np.sqrt(2)

Usplit_fixed[8][8] = 0
Usplit_fixed[12][12] = 0
Usplit_fixed[32][32] = 0
Usplit_fixed[48][48] = 0

Usplit_fixed[63-8][63-8] = 0
Usplit_fixed[63-12][63-12] = 0
Usplit_fixed[63-32][63-32] = 0
Usplit_fixed[63-48][63-48] = 0


Usplit_fixed[2][8] = 1
Usplit_fixed[3][12] = 1
Usplit_fixed[8][32] = 1
# Usplit_fixed[31][15] = 1
# Usplit_fixed[2][32] = 1
# Usplit_fixed[32][31] = 1
# Usplit_fixed[32][15] = 1
# Usplit_fixed[31][15] = 1
# Usplit_fixed[32][15] = 1
# Usplit_fixed[2][32] = 1
# Usplit_fixed[3][48] = 1


Usplit_fixed[63-2][63-8] = 1
Usplit_fixed[63-3][63-12] = 1
Usplit_fixed[63-8][63-32] = 1
# Usplit_fixed[63-31][63-15] = 1
# Usplit_fixed[63-2][63-32] = 1
# Usplit_fixed[63-32][63-31] = 1
# Usplit_fixed[63-32][63-15] = 1
# Usplit_fixed[63-31][63-15] = 1
# Usplit_fixed[63-32][63-15] = 1
# Usplit_fixed[63-2][63-32] = 1
# Usplit_fixed[63-3][63-48] = 1

u = np.matrix(Usplit_fixed)

ut = np.matrix(Usplit_fixed).getH()

a = np.matrix(Usplit_fixed).getH() * np.matrix(Usplit_fixed)

# print(a.tolist())

# for i in range(1):
#     print(i)

#     # print(ut[i,:].reshape())
#     # a = np.reshape(ut[i , :], 64)
#     # b = np.reshape(u[: , i], 64)

#     # print(a)
#     # print(b)
#     # print("\n")

#     # print(u[:,i].tolist())
#     # print("DOT: ", np.dot(a, b))
#     print("DOT: ", np.dot(ut[i, :], u[:, i]))
#     # assert np.dot(ut[i, :], u[:, i]) == [[1]]

for i in range(64):
    print(i, u.tolist()[i])

print("\n")

for i in range(64):
    print(i, ut.tolist()[i])

print("\n")

for i in range(64):
    print(i, a.tolist()[i])
    # print(i, Usplit_fixed[i])

# for i in range(64):
    # print(i, list(a[i]))

0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
3 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0

#### Example of Split operation and then merge on a 3x3 board. This is a simple example to ensure that these gates do work correctly.

In [165]:
q2 = QuantumRegister(4, "box")
c2 = ClassicalRegister(4)
bell = QuantumCircuit(q2, c2)
bell.x(0)
target_state_bell = qi.Statevector.from_instruction(bell.reverse_bits()).to_dict()
print(target_state_bell)

bell.unitary(Usplit,[0,1,2])
bell.draw()

{'1000': (1+0j)}


┌───┐┌──────────┐
box_0: ┤ X ├┤0         ├
       └───┘│          │
box_1: ─────┤1 Unitary ├
            │          │
box_2: ─────┤2         ├
            └──────────┘
box_3: ─────────────────
                        
 c6: 4/═════════════════

In [166]:
target_state_bell = qi.Statevector.from_instruction(bell.reverse_bits()).to_dict()
print(target_state_bell)

{'0010': 0.7071067811865475j, '0100': 0.7071067811865475j}


In [167]:
bell.unitary(Umerge,[3,2,1])
bell.draw()

┌───┐┌──────────┐            
box_0: ┤ X ├┤0         ├────────────
       └───┘│          │┌──────────┐
box_1: ─────┤1 Unitary ├┤2         ├
            │          ││          │
box_2: ─────┤2         ├┤1 Unitary ├
            └──────────┘│          │
box_3: ─────────────────┤0         ├
                        └──────────┘
 c6: 4/═════════════════════════════

In [168]:
# target_state_bell = qi.Statevector.from_instruction(bell)
target_state_bell = qi.Statevector.from_instruction(bell.reverse_bits()).to_dict()
print(target_state_bell)

{'0001': (0.9999999999999998+0j), '0010': -2.2371143170757382e-17j}


In [169]:
bell.measure([0,1,2,3], [3,2,1,0])
from qiskit import execute, Aer
job = execute(bell,Aer.get_backend('qasm_simulator'),shots=1024)
counts = job.result().get_counts(bell)
print(counts) # counts is a dictionary

{'0001': 1024}


### State of the Board:

The state of the board is being created here.

The state of the board consists of (number of boxes * 2) qubits, meaning each box in the grid is represented in 2 qubits. The first qubit represents the occupancy of the box, 1 in case of being occupied and 0 if not occupied. The second qubit will represent the color of the piece in it, 0 if white and 1 if black.

The state will store every alternate box only since every other alternate box is unused in the game of checkers, i.e the piece cannot move there.

The game will simulate a 6x6 checkerboard meaning 36 qubits will be stored in the state.


In [170]:
import pprint

board = []
c_board_regs = []
pos_dic = {}   

box_to_qubit = []

white_occ = [(0,0), (0,2), (0, 4), (1,1), (1,3), (1, 5)]
black_occ = [(4,1), (4,3), (4, 5), (5,0), (5,2), (5, 4)]

remove = [(1,0), (3,0), (5,0), (0,1), (2,1), (4,1), (1,2), (3,2), (5,2), (0,3), (2,3), (4,3), (1,4), (3,4), (5,4), (0,5), (2,5), (4,5)]

def init():
    
    for i in range(6):
        
        temp = []

        for j in range(6):

            pos_dic[f'box_({i},{j})'] = i, j
            
            if (j,i) not in remove:
                qreg = QuantumRegister(2, name=f'box_({j},{i})')
                c_board_regs.append(ClassicalRegister(2, name=f'cbox_({j},{i})'))
                box_to_qubit.append((j,i))
                box_to_qubit.append((j,i))

                temp.append(qreg)       
        board.append(temp)

init()

temp = []
for i in board:
    temp += i
   
# print(temp)
q_board = QuantumCircuit(*temp, QuantumRegister(1, "ancilla"), *c_board_regs)


for i in range(0, 12, 2):
    q_board.x(i)

for i in range(35, 23, -2):
    q_board.x(i)
    q_board.x(i-1)
    

q_board.barrier()

q_board.draw()



┌───┐ ░ 
 box_(0,0)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(0,0)_1: ──────░─
              ┌───┐ ░ 
 box_(2,0)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(2,0)_1: ──────░─
              ┌───┐ ░ 
 box_(4,0)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(4,0)_1: ──────░─
              ┌───┐ ░ 
 box_(1,1)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(1,1)_1: ──────░─
              ┌───┐ ░ 
 box_(3,1)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(3,1)_1: ──────░─
              ┌───┐ ░ 
 box_(5,1)_0: ┤ X ├─░─
              └───┘ ░ 
 box_(5,1)_1: ──────░─
                    ░ 
 box_(0,2)_0: ──────░─
                    ░ 
 box_(0,2)_1: ──────░─
                    ░ 
 box_(2,2)_0: ──────░─
                    ░ 
 box_(2,2)_1: ──────░─
                    ░ 
 box_(4,2)_0: ──────░─
                    ░ 
 box_(4,2)_1: ──────░─
                    ░ 
 box_(1,3)_0: ──────░─
                    ░ 
 box_(1,3)_1: ──────░─
                    ░ 
 box_(3,3)_0: ──────░─
                    ░ 
 box_(3,3)_1: ──────░─
                    ░ 
 box_(5,3)_0: ──────░─
                    ░ 
 box_(5,3)_1: ──────░─
              ┌───┐ ░ 
 box_(0,4)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(0,4)_1: ┤ X ├─░─
              ├───┤ ░ 
 box_(2,4)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(2,4)_1: ┤ X ├─░─
              ├───┤ ░ 
 box_(4,4)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(4,4)_1: ┤ X ├─░─
              ├───┤ ░ 
 box_(1,5)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(1,5)_1: ┤ X ├─░─
              ├───┤ ░ 
 box_(3,5)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(3,5)_1: ┤ X ├─░─
              ├───┤ ░ 
 box_(5,5)_0: ┤ X ├─░─
              ├───┤ ░ 
 box_(5,5)_1: ┤ X ├─░─
              └───┘ ░ 
   ancilla_0: ──────░─
                    ░ 
cbox_(0,0): 2/════════
                      
cbox_(2,0): 2/════════
                      
cbox_(4,0): 2/════════
                      
cbox_(1,1): 2/════════
                      
cbox_(3,1): 2/════════
                      
cbox_(5,1): 2/════════
                      
cbox_(0,2): 2/════════
                      
cbox_(2,2): 2/════════
                      
cbox_(4,2): 2/════════
                      
cbox_(1,3): 2/════════
                      
cbox_(3,3): 2/════════
                      
cbox_(5,3): 2/════════
                      
cbox_(0,4): 2/════════
                      
cbox_(2,4): 2/════════
                      
cbox_(4,4): 2/════════
                      
cbox_(1,5): 2/════════
                      
cbox_(3,5): 2/════════
                      
cbox_(5,5): 2/════════

In [171]:
pos_dic = {}
# lst_x = ["a", "b", "c", "d", "e", "f"]
# lst_y = ["0", "1", "2", "3", "4", "5"]

for i in range(6):
    for j in range(6):
        pos_dic[f'box_({i},{j})'] = i, j



### Gates

We have three unitaries for the operations of the jump, split jump and merge. 

The jump unitary is a swap operator that acts on two inputs of two qubits each. There is the target box and the source box each of which holds two qubits to describe its state completely. We apply our jump unitary to swap the states of the target and source.

The split unitary works on three inputs/boxes. There are two target boxes and one source box each of which is composed of 2 qubits each. It allows the player to move their piece into a superposition. The split unitary removes the occupancy and color qubit of the source box and creates two superpositions in the state, each of which contains the move in a single direction. For e.g: one superposition will contain the state if the piece jumped in the left diagonal and one superposition will contain the state if the piece jumped in the right diagonal.

The merge unitary works on three inputs/boxes as well. There are two source boxes and one target box. It is the dagger of the split unitary. It allows the player to transfer their superposition piece back into classical state. It sets the occupancy and color qubit of the source boxes to 0 and sets the occupancy qubit of target box to 1 and color qubit accordingly.

## Moves
- Jump Unitary


    ![jump](jump.png)



$\newline$
- Split


     ![split](split.png)




$\newline$
- Merge


    ![merge](merge.png)

In [172]:
def jump(src, target, qc):
    for i in range(2):
        qc.unitary(Ujump, [src[i], target[i]])

def split(src, target1, target2, qc):
    # for i in range(2):
    #     qc.unitary(Usplit, [src[i], target1[i], target2[i]])
    # qc.unitary(Usplit, [src[1], target1[1], target2[1]])
    qc.unitary(Usplit, [src[0], target1[0], target2[0]])
    # qc.cx(1, src[1])
    # qc.cx(src[1], target1[1])
    # qc.cx(src[1], target2[1])

    # qc.unitary(Usplit_fixed, [src[0], src[1], target1[0], target1[1], target2[0], target2[1]])

def merge(src1, src2, target, qc):
    qc.unitary(Umerge, [src1[0], src2[0], target[0]])
    qc.unitary(Umerge, [src1[1], src2[1], target[1]])


# def print_state_board(counts):


In [178]:
def get_diag_white(box):
    
    return (pos_dic[box][0]+1, pos_dic[box][1]+1), (pos_dic[box][0]-1, pos_dic[box][1]+1)

def get_diag_black(box):

    return (pos_dic[box][0]+1, pos_dic[box][1]-1), (pos_dic[box][0]-1, pos_dic[box][1]-1)


src = 'box_(1,1)'
diag1, diag2 = get_diag_white('box_(1,1)')

# print(diag1, diag2)
# print(diag1)
# print(box_to_qubit.index(diag1))
src_bits = (box_to_qubit.index((1,1)), box_to_qubit.index((1,1))+1)
target1_bits = (box_to_qubit.index(diag1), box_to_qubit.index(diag1) + 1)
target2_bits = (box_to_qubit.index(diag2), box_to_qubit.index(diag2) + 1)

# jump(src_bits, target1_bits, q_board)
# split(src_bits, target1_bits, target2_bits, q_board)
# q_board.cx(src_bits[1], 36)
# q_board.swap(src_bits[1], target1_bits[1])
# q_board.swap(36, target2_bits[1])

src = 'box_(4,4)'
diag1, diag2 = get_diag_black('box_(4,4)')

src_bits = (box_to_qubit.index((4,4)), box_to_qubit.index((4,4))+1)
target1_bits = (box_to_qubit.index(diag1), box_to_qubit.index(diag1) + 1)
target2_bits = (box_to_qubit.index(diag2), box_to_qubit.index(diag2) + 1)

# jump(src_bits, target1_bits, q_board)

split(src_bits, target1_bits, target2_bits, q_board)
q_board.cx(src_bits[1], 36)
q_board.swap(src_bits[1], target1_bits[1])
q_board.swap(36, target2_bits[1])


# merge(target1_bits, target2_bits, src_bits, q_board)

#qi.Statevector.from_instruction(q_board.reverse_bits()).to_dict()

# q_board.draw()
q_board.measure([i for i in range(36)], [i for i in range(35, -1, -1)])
job = execute(q_board,Aer.get_backend('qasm_simulator'),shots=256)
counts = job.result().get_counts(q_board)
print(counts) # counts is a dictionary


{'10 10 10 00 10 10 00 10 00 00 01 11 11 11 00 11 11 11': 50, '10 10 10 00 10 10 10 00 00 00 01 11 11 11 00 11 11 11': 60, '10 10 10 00 10 10 10 00 00 00 11 01 11 11 00 11 11 11': 78, '10 10 10 00 10 10 00 10 00 00 11 01 11 11 00 11 11 11': 68}


In [174]:
def capture(target, source):
   qc = QuantumCircuit(4)

   

   qc.cx(target[0], source[0])
   
   qc.cx(target[1], source[1])

   target_state_bell = qi.Statevector.from_instruction(qc.reverse_bits()).to_dict()
   print(target_state_bell)


In [175]:
def get_box_qubits(box_num):
    return box_num * 2, (box_num * 2) + 1


def apply_Usplit(qc, source_box, target1_box, target2_box):
    src = get_box_qubits(source_box)
    t1 = get_box_qubits(target1_box)
    t2 = get_box_qubits(target2_box)
    
    print(src, t1, t2)
    for bit in range(2):
        # print([src[bit], t1[bit], t2[bit]])
        qc.unitary(Usplit, [src[bit], t1[bit], t2[bit]])
        # print(qi.Statevector.from_instruction(q_circuit.reverse_bits()).to_dict())
    
# def print(state):
    
        

In [176]:
# Making circuit of 3x3 boxes
q_registers = []

for i in range(4):
    q_registers.append(QuantumRegister(2, f"box_{i}"))

q_circuit = QuantumCircuit(*q_registers)

# Placing player 1 at box 1
qubits = get_box_qubits(1)
q_circuit.x(qubits[0])
q_circuit.x(qubits[1])

# qi.Statevector.from_instruction(q_circuit.reverse_bits()).to_dict()
q_circuit.draw()


box_0_0: ─────
              
box_0_1: ─────
         ┌───┐
box_1_0: ┤ X ├
         ├───┤
box_1_1: ┤ X ├
         └───┘
box_2_0: ─────
              
box_2_1: ─────
              
box_3_0: ─────
              
box_3_1: ─────

In [177]:
# Split jump to box 2 and 3

apply_Usplit(q_circuit, source_box=1, target1_box=2, target2_box=3)
qi.Statevector.from_instruction(q_circuit.reverse_bits()).to_dict()

(2, 3) (4, 5) (6, 7)


{'00000011': (-0.4999999999999999+0j),
 '00000110': (-0.4999999999999999+0j),
 '00001001': (-0.4999999999999999+0j),
 '00001100': (-0.4999999999999999+0j)}

## Updated jump with pathing consideration

We apply a different type of Unitary called the slide unitary:

 ![jump](split_slide.png)
